In [2]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
from collections import defaultdict

In [6]:
fields = ['package','forks','stars','so_tag_counts','so_question_count','downloads','growth_rates']
DF = pd.read_csv('../output/JS_VIZ_data.csv', usecols=fields)

In [ ]:
DF.sort_values(['so_questions_counts'],ascending=False).head()

In [7]:
#take out all packages with any nan values and put them in their own DF
nanDF = DF[pd.isnull(DF).any(axis=1)]

DF.dropna(axis=0, how='any', inplace=True)

In [9]:
#scale the values, such that for each metric: mean = 0 and std = 1
#use the same scaling for the nanDF dataset
metrics = ['forks','stars','so_tag_counts','so_question_count','downloads','growth_rates']
scaled_DF = pd.DataFrame(columns=metrics)
scaled_NaN = pd.DataFrame(columns=metrics)

for metric in metrics:
    mean = DF[metric].mean()
    std = DF[metric].std()
    scaled_DF[metric] = DF[metric].apply(lambda x: (x-mean)/std)
    scaled_NaN[metric] = nanDF[metric].apply(lambda x: (x-mean)/std)

scaled_DF['package']= DF['package']
scaled_NaN['package'] = nanDF['package']

In [10]:
#merge forks and stars to one GitHub metric, Tags and Questions to Stack Overflow, downloads and dl growth to Downloads
#overall is sum of all scaled metrics
final_DF = pd.DataFrame(columns=['Package', 'Rank', 'Overall', 'Github', 'Stack Overflow', 'Downloads'])
final_DF['Package'] = scaled_DF['package']
final_DF['Github'] = scaled_DF[['forks', 'stars']].mean(axis=1)
final_DF['Stack Overflow'] = scaled_DF[['so_tag_counts', 'so_question_count']].mean(axis=1)
final_DF['Downloads'] = scaled_DF[['downloads','growth_rates']].mean(axis=1)
final_DF['Overall'] = final_DF[['Github','Stack Overflow','Downloads']].sum(axis=1)
final_DF['Rank'] = final_DF['Overall'].rank(ascending=0).astype(int)

In [11]:
#combine nanDF metrics to be added to the bottom fo the DF
final_nan_DF = pd.DataFrame(columns=['Package', 'Github', 'Stack Overflow', 'Downloads'])
final_nan_DF['Package'] = scaled_NaN['package']
final_nan_DF['Github'] = scaled_NaN[['forks', 'stars']].mean(axis=1)
final_nan_DF['Stack Overflow'] = scaled_NaN[['so_tag_counts', 'so_question_count']].mean(axis=1)
final_nan_DF['Downloads'] = scaled_NaN[['downloads','growth_rates']].mean(axis=1)

ranked_data = final_DF.append(final_nan_DF)

In [12]:
ranked_data.sort_values(['Overall'], axis=0, ascending=False, inplace=True)
ranked_data = ranked_data.reindex_axis(['Package','Rank','Overall','Github','Stack Overflow','Downloads'], axis=1)

In [13]:
ranked_data.style.set_table_styles([
    {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}
])

In [ ]:
ranked_data.to_csv('../output/js_viz_final_Rankings.csv', index = False)